In [ ]:
import numpy as np
import scipy.linalg as la
import pickle

from scipy.optimize import minimize
from itertools import product

from qulacs.gate import CNOT, RX, RY, RZ
from qulacsvis import circuit_drawer

from qiskit import *
from qiskit.quantum_info.states.random import random_density_matrix
from qiskit.quantum_info import Statevector, DensityMatrix, partial_trace

import matplotlib.pyplot as plt
from mpl_axes_aligner import shift
plt.rcParams.update(
    {"text.usetex": True, "font.family": "serif", "font.size": 10}
)

In [ ]:
def dictionary_of_actions(num_qubits):
    dictionary = dict()
    i = 0
    for c, x in product(range(num_qubits),
                        range(1, num_qubits)):
        dictionary[i] =  [c, x, num_qubits, 0]
        i += 1
    for r, h in product(range(num_qubits),
                           range(1, 4)):
        dictionary[i] = [num_qubits, 0, r, h]
        i += 1
    return dictionary

def make_circuit_qiskit(circuit, action, qubits):
    ctrl = action[0]
    targ = (action[0] + action[1]) % qubits
    rot_qubit = action[2]
    rot_axis = action[3]
    if ctrl < qubits:
        circuit.cx([ctrl], [targ])
    if rot_qubit < qubits:
        if rot_axis == 1:
            circuit.rx(0, rot_qubit)
        elif rot_axis == 2:
            circuit.ry(0, rot_qubit)
        elif rot_axis == 3:
            circuit.rz(0, rot_qubit)
        else:
            stop
    return circuit

In [ ]:
qubits, rank, seed, total_agent_seed = 2,4,5,102
episodes = 10000
test_train = 'train'
method_list = ['bs-dqn', 'ddqn']
method = method_list[1]
print(method)
for agent_seed in range(100, total_agent_seed+1):
    print(agent_seed, method)
    if method == 'bs-dqn':
        data = np.load(f'results/global_COBYLA/h_s_{qubits}_rank_{rank}_{seed}_bs/summary_{agent_seed}.npy',allow_pickle=True)[()]
    elif method == 'ddqn':
        data = np.load(f'results/global_COBYLA/h_s_{qubits}_rank_{rank}_{seed}/summary_{agent_seed}.npy',allow_pickle=True)[()]
        
    succ_ep_info_dict = {}
    success_ep, success_ep_error = [], []
    average_err = 0
    for ep_no in range(episodes):
        data_error_last = data[test_train][ep_no]['errors'][-1]
    #     data_error_last -= 6.694750006687696
        if data_error_last <= 1e-5:
            success_ep.append(ep_no)
            success_ep_error.append(data_error_last)
            average_err += data_error_last
    average_err /= len(success_ep)
    print(f'First successful episode: {success_ep[0]} and error {round(success_ep_error[0],6)}, total successful episode: {len(success_ep)}')

    x = [data[test_train][ep_no]['actions'] for ep_no in success_ep]
    circuit_info_dict = {}
    depth_list = []
    oneq_gate_list = []
    twoq_gate_list = []
    gate_num_list = []

    for succ_ep in success_ep:
        actions = data[test_train][succ_ep]['actions']
        circuit = QuantumCircuit(qubits)
        for a in actions:
            action = dictionary_of_actions(qubits)[a]
            final_circuit = make_circuit_qiskit(circuit, action, qubits)

        gate_info = final_circuit.count_ops()
        key_list = gate_info.keys()
        one_gate, two_gate = 0,0
        for k in key_list:
            if k == 'cx':
                two_gate += gate_info[k]
            else:
                one_gate += gate_info[k]

        twoq_gate_list.append(two_gate)
        oneq_gate_list.append(one_gate)
        gate_num_list.append(one_gate+two_gate)        
        depth_list.append(final_circuit.depth())

    circuit_info_dict['depth_list'] = depth_list
    circuit_info_dict['oneq_gate_list'] = oneq_gate_list
    circuit_info_dict['twoq_gate_list'] = twoq_gate_list
    circuit_info_dict['gate_num_list'] = gate_num_list
    circuit_info_dict['succ_ep_list'] = success_ep
    circuit_info_dict['succ_ep_err_list'] = success_ep_error
    print(circuit_info_dict.keys())
    if method == 'bs-dqn':
        with open(f'plot_data/{qubits}_qubit_mixed_rank_{rank}_seed_{seed}_agent_seed_{agent_seed}_data_bs.p', 'wb') as fp:
            pickle.dump(circuit_info_dict, fp)
    elif method == 'ddqn':
        print(1111)
        with open(f'plot_data/{qubits}_qubit_mixed_rank_{rank}_seed_{seed}_agent_seed_{agent_seed}_data.p', 'wb') as fp:
            pickle.dump(circuit_info_dict, fp)

In [ ]:
qubits, rank, seed, agent_seed_list = 2, 4, 5, [100, 101, 102]

plot_data_list = []
plot_data_err_list = []

for method in method_list:
    min_depth_list = []
    ave_depth_list = []
    min_gate_list = []
    ave_gate_list = []
    for agent_seed in agent_seed_list:
        data = []
        if method == 'bs-dqn':
            file = f"plot_data/{qubits}_qubit_mixed_rank_{rank}_seed_{seed}_agent_seed_{agent_seed}_data_bs.p"
        elif method == 'ddqn':
            file = f"plot_data/{qubits}_qubit_mixed_rank_{rank}_seed_{seed}_agent_seed_{agent_seed}_data.p"
        with (open(file, "rb")) as openfile:
            while True:
                try:
                    data.append(pickle.load(openfile))
                except EOFError:
                    break
        data_dict = data[0]
        min_depth = np.min(data_dict['depth_list'])
        ave_depth = np.mean(data_dict['depth_list'])
        ave_rot, ave_cx = np.mean(data_dict['oneq_gate_list']), np.mean(data_dict['twoq_gate_list'])
        min_gate, ave_gate = np.min(data_dict['gate_num_list']), np.mean(data_dict['gate_num_list'])

        min_depth_list.append(min_depth)
        ave_depth_list.append(ave_depth)
        min_gate_list.append(min_gate)
        ave_gate_list.append(ave_gate)
    
    plot_data_list.append(np.mean(min_depth_list))
    plot_data_list.append(np.mean(ave_depth_list))
    plot_data_list.append(np.mean(min_gate_list))
    plot_data_list.append(np.mean(ave_gate_list))
    
    plot_data_err_list.append(np.std(min_depth_list))
    plot_data_err_list.append(np.std(ave_depth_list))
    plot_data_err_list.append(np.std(min_gate_list))
    plot_data_err_list.append(np.std(ave_gate_list))

In [ ]:
X = ['Minimum \n depth', 'Average \n depth', 'Minimum \n gate count', 'Average \n gate count']
X_axis = np.arange(len(X))
width = 0.3
plt.bar( X_axis, plot_data_list[:4],width, 
        yerr=plot_data_err_list[:4], alpha=0.7, 
        ecolor='black', capsize=10, label = 'Bootstrap-DDQN')

plt.bar( X_axis+width, plot_data_list[4:],width, 
        yerr=plot_data_err_list[4:], alpha=0.7, 
        ecolor='black', capsize=10, label = 'DDQN')

plt.xticks(X_axis+0.47*width, X)
plt.legend(ncol=2, bbox_to_anchor=(0.8, 1.12))
# plt.tight_layout()
plt.savefig('plot/rl_method_comparison.pdf')
plt.savefig('plot/rl_method_comparison.png')